In [ ]:
# ref: the soil dataset used in balance tree
# https://msystems.asm.org/content/2/1/e00162-16

In [1]:
import sys
import biom
from biom.util import biom_open
import pandas as pd
import numpy as np

In [2]:
def biom2pandas(file_biom, withTaxonomy=False, astype=int):
    """ Converts a biom file into a Pandas.DataFrame
    Parameters
    ----------
    file_biom : str
        The path to the biom file.
    withTaxonomy : bool
        If TRUE, returns a second Pandas.Series with lineage information for
        each feature, e.g. OTU or deblur-sequence. Default: FALSE
    astype : type
        datatype into each value of the biom table is casted. Default: int.
        Use e.g. float if biom table contains relative abundances instead of
        raw reads.
    Returns
    -------
    A Pandas.DataFrame holding holding numerical values from the biom file.
    If withTaxonomy is TRUE then a second Pandas.DataFrame is returned, holding
    lineage information about each feature.
    Raises
    ------
    IOError
        If file_biom cannot be read.
    ValueError
        If withTaxonomy=TRUE but biom file does not hold taxonomy information.
    """
    try:
        table = biom.load_table(file_biom)
        counts = pd.DataFrame(table.matrix_data.T.todense().astype(astype),
                              index=table.ids(axis='sample'),
                              columns=table.ids(axis='observation')).T
        if withTaxonomy:
            try:
                md = table.metadata_to_dataframe('observation')
                levels = [col
                          for col in md.columns
                          if col.startswith('taxonomy_')]
                if levels == []:
                    raise ValueError(('No taxonomy information found in '
                                      'biom file.'))
                else:
                    taxonomy = md.apply(lambda row:
                                        ";".join([row[l] for l in levels]),
                                        axis=1)
                    return counts, taxonomy
            except KeyError:
                raise ValueError(('Biom file does not have any '
                                  'observation metadata!'))
        else:
            return counts
    except IOError:
        raise IOError('Cannot read file "%s"' % file_biom)


def pandas2biom(file_biom, table, taxonomy=None, err=sys.stderr):
    """ Writes a Pandas.DataFrame into a biom file.
    Parameters
    ----------
    file_biom: str
        The filename of the BIOM file to be created.
    table: a Pandas.DataFrame
        The table that should be written as BIOM.
    taxonomy : pandas.Series
        Index is taxons corresponding to table, values are lineage strings like
        'k__Bacteria; p__Actinobacteria'
    err : StringIO
        Stream onto which errors / warnings should be printed.
        Default is sys.stderr
    Raises
    ------
    IOError
        If file_biom cannot be written.
    TODO
    ----
        1) also store taxonomy information
    """
    try:
        bt = biom.Table(table.values,
                        observation_ids=table.index,
                        sample_ids=table.columns)

        # add taxonomy metadata if provided, i.e. is not None
        if taxonomy is not None:
            if not isinstance(taxonomy, pd.core.series.Series):
                raise AttributeError('taxonomy must be a pandas.Series!')
            idx_missing_intable = set(table.index) - set(taxonomy.index)
            if len(idx_missing_intable) > 0:
                err.write(('Warning: following %i taxa are not in the '
                           'provided taxonomy:\n%s\n') % (
                          len(idx_missing_intable),
                          ", ".join(idx_missing_intable)))
                missing = pd.Series(
                    index=idx_missing_intable,
                    name='taxonomy',
                    data='k__missing_lineage_information')
                taxonomy = taxonomy.append(missing)
            idx_missing_intaxonomy = set(taxonomy.index) - set(table.index)
            if (len(idx_missing_intaxonomy) > 0) and err:
                err.write(('Warning: following %i taxa are not in the '
                           'provided count table, but in taxonomy:\n%s\n') % (
                          len(idx_missing_intaxonomy),
                          ", ".join(idx_missing_intaxonomy)))

            t = dict()
            for taxon, linstr in taxonomy.iteritems():
                # fill missing rank annotations with rank__
                orig_lineage = {annot[0].lower(): annot
                                for annot
                                in (map(str.strip, linstr.split(';')))}
                lineage = []
                for rank in settings.RANKS:
                    rank_char = rank[0].lower()
                    if rank_char in orig_lineage:
                        lineage.append(orig_lineage[rank_char])
                    else:
                        lineage.append(rank_char+'__')
                t[taxon] = {'taxonomy': ";".join(lineage)}
            bt.add_metadata(t, axis='observation')

        with biom_open(file_biom, 'w') as f:
            bt.to_hdf5(f, "example")
    except IOError:
        raise IOError('Cannot write to file "%s"' % file_biom)

### Balance_88soils

In [19]:
soils_biom = biom2pandas('../88soils/238_otu_table.biom')
soils_biom.shape

(7396, 89)

In [20]:
soils_biom.head(3)

,103.CA2,103.CO3,103.SR3,103.IE2,103.BP1,103.VC2,103.SA2,103.GB2,103.CO2,103.KP1,...,103.LQ1,103.HI1,103.RT1,103.HI2,103.DF1,103.CF3,103.AR1,103.TL1,103.HI4,103.BB1
1124701,15,14,1,8,13,7,6,3,2,2,...,0,0,0,0,0,0,0,0,0,0
244336,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
973124,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [21]:
soils_taxa = pd.read_csv('../88soils/88soils_taxonomy.txt', sep='\t', index_col='Feature ID')
soils_taxa.shape

(7396, 1)

In [22]:
soils_taxa.head(3)

,Taxon
Feature ID,
1000512,k__Bacteria;p__Actinobacteria;c__Thermoleophil...
1000547,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...
1000654,k__Bacteria;p__Bacteroidetes;c__Sphingobacteri...


In [23]:
taxa_new = soils_taxa.Taxon.str.split(pat=";", expand=True)
taxa_new.head(5)
# ref: https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/

,0,1,2,3,4,5,6
Feature ID,,,,,,,
1000512,k__Bacteria,p__Actinobacteria,c__Thermoleophilia,o__Gaiellales,f__Gaiellaceae,g__,s__
1000547,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,s__
1000654,k__Bacteria,p__Bacteroidetes,c__Sphingobacteriia,o__Sphingobacteriales,f__Sphingobacteriaceae,g__,s__
1000757,k__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhizobiales,f__Bradyrhizobiaceae,g__,s__
1000876,k__Bacteria,p__Actinobacteria,c__Actinobacteria,o__Actinomycetales,f__Nocardioidaceae,g__Nocardioides,s__


In [24]:
soils_taxa['Genus'] = taxa_new[5]
soils_taxa.head(5)

,Taxon,Genus
Feature ID,,
1000512,k__Bacteria;p__Actinobacteria;c__Thermoleophil...,g__
1000547,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,g__Streptococcus
1000654,k__Bacteria;p__Bacteroidetes;c__Sphingobacteri...,g__
1000757,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__
1000876,k__Bacteria;p__Actinobacteria;c__Actinobacteri...,g__Nocardioides


In [26]:
soils_taxa.Genus.value_counts()

g__                           5213
g__Rhodoplanes                 144
g__Bacillus                    110
g__Candidatus Solibacter       100
g__Flavobacterium               71
                              ... 
g__Rhodocyclus                   1
g__Marinobacter                  1
g__Afipia                        1
g__Candidatus Amoebophilus       1
g__Desulfotomaculum              1
Name: Genus, Length: 335, dtype: int64

In [27]:
# only keep those with genus assignment
soils_taxa_sub = soils_taxa[soils_taxa.Genus != 'g__']
soils_taxa_sub.shape

(2183, 2)

In [28]:
soils_taxa_sub.head(3)

,Taxon,Genus
Feature ID,,
1000547,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,g__Streptococcus
1000876,k__Bacteria;p__Actinobacteria;c__Actinobacteri...,g__Nocardioides
1003206,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Sphingomonas


In [39]:
# partition biom table 
soils_biom.index = soils_biom.index.astype('int64') 
soils_biom_sub = soils_biom.merge(soils_taxa_sub, how='inner', left_index=True, right_index=True)
soils_biom_sub.shape

(2183, 91)

In [40]:
soils_biom_sub.head(3)

,103.CA2,103.CO3,103.SR3,103.IE2,103.BP1,103.VC2,103.SA2,103.GB2,103.CO2,103.KP1,...,103.RT1,103.HI2,103.DF1,103.CF3,103.AR1,103.TL1,103.HI4,103.BB1,Taxon,Genus
244336,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,g__Paenibacillus
809489,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,g__Bacillus
533625,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,k__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g__Novosphingobium


In [41]:
soils_biom_sub.set_index('Taxon', inplace=True)
soils_biom_sub.drop(['Genus'], axis=1, inplace=True)

In [42]:
soils_biom_sub.shape

(2183, 89)

In [43]:
soils_biom_sub.head(3)

,103.CA2,103.CO3,103.SR3,103.IE2,103.BP1,103.VC2,103.SA2,103.GB2,103.CO2,103.KP1,...,103.LQ1,103.HI1,103.RT1,103.HI2,103.DF1,103.CF3,103.AR1,103.TL1,103.HI4,103.BB1
Taxon,,,,,,,,,,,,,,,,,,,,,
k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Paenibacillaceae;g__Paenibacillus;s__,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;g__Bacillus;s__muralis,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Sphingomonadales;f__Sphingomonadaceae;g__Novosphingobium;s__,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# transpose the dataframe 
soils_biom_sub_t = soils_biom_sub.T
soils_biom_sub_t.shape

(89, 2183)

In [45]:
soils_biom_sub_t.head(3)

Taxon,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Paenibacillaceae;g__Paenibacillus;s__,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;g__Bacillus;s__muralis,k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Sphingomonadales;f__Sphingomonadaceae;g__Novosphingobium;s__,k__Bacteria;p__Acidobacteria;c__Solibacteres;o__Solibacterales;f__Solibacteraceae;g__Candidatus Solibacter;s__,k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Hyphomicrobiaceae;g__Rhodoplanes;s__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Nocardioidaceae;g__Nocardioides;s__,k__Bacteria;p__Bacteroidetes;c__Flavobacteriia;o__Flavobacteriales;f__Flavobacteriaceae;g__Flavobacterium;s__,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Peptococcaceae;g__Desulfotomaculum;s__,k__Bacteria;p__Acidobacteria;c__Acidobacteriia;o__Acidobacteriales;f__Koribacteraceae;g__Candidatus Koribacter;s__,k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;f__Comamonadaceae;g__Methylibium;s__,...,k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;f__Comamonadaceae;g__Rhodoferax;s__,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Xanthomonadales;f__Xanthomonadaceae;g__Dokdonella;s__,k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__,k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Methylophilales;f__Methylophilaceae;g__Methylotenera;s__mobilis,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Streptomycetaceae;g__Streptomyces;s__,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Xanthomonadales;f__Xanthomonadaceae;g__Luteimonas;s__,k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Hyphomicrobiaceae;g__Rhodoplanes;s__,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Planococcaceae;g__Solibacillus;s__,k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;f__Comamonadaceae;g__Ramlibacter;s__,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Mycobacteriaceae;g__Mycobacterium;s__
103.CA2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103.CO3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
103.SR3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1


In [46]:
# make sure that each genus exist in at least one sample
soils_biom_sub_t.sum(axis=1).describe() # column sum

count     89.000000
mean     275.932584
std      115.508244
min        1.000000
25%      213.000000
50%      254.000000
75%      319.000000
max      805.000000
dtype: float64

In [47]:
# export
soils_biom_sub_t.to_csv('../88soils/88soils_genus_table.txt', sep='\t')

In [48]:
# check that not rarefied
soils_biom_sub_t.sum(axis=0).describe() # row sum

count    2183.000000
mean       11.249656
std        33.869668
min         1.000000
25%         1.000000
50%         3.000000
75%         8.000000
max       690.000000
dtype: float64